In [16]:
from pprint import pprint
import pymysql
import pandas as pd

# 순서

1. SQL 에서 데이터 가져오고 DF로 만들기

2. 리뷰 문장구분 & feature 매칭

3. KW 사전 구축하기

4. KW로 SS / CS 구분하기

5. MPQA 사전으로 POS, NEG 라벨링

계속 순서대로 돌면서 코드 수정 & 보완할 것

앞으로 해야 할 것: CS에서 PRODUCT NAME 포함한 CS만 남기기

## 1. SQL 에서 데이터 가져오고 DF로 만들기

In [45]:

db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = """SELECT Review FROM Reviews2 LIMIT 20000"""
data = pd.read_sql(SQL, db)
data

,Review
0,"I've had this camera for years now.... (5-7), ..."
1,... after two previous attempts at buying my m...
2,I like this camera so much that I bought two m...
3,I got the HP R707 shortly after it was introdu...
4,This camera was a used camera so I was concern...
5,I've already had an HP Photosmart R707 Digital...
6,This is second camera of this model that I hav...
7,HP Photosmart R707: if you're looking to go be...
8,This is a wonderful digital camera for beginne...
9,It is really probably just my fault for not re...


## 2. 리뷰 문장구분 & feature 매칭

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seonmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### 앞으로 추가해야함

In [47]:
feature={
    '12': ['resolution', 'pixel', 'megapixel'],   # 0
    '13': ['lens','wide angle','normal range'],   # 1
    '14': ['optical','zoom','optical zoom','digital zoom'],  # 2
    '15': ['memory','megabytes','MB'],
    '16': ['burst','continuous','shutter','recovery','motion','sport'],
    '17': ['battery','batteries','power'],
    '18': ['focus','exposure','manual','iso'],
    '19': ['LCD','screen'],
    '20': ['compression','compress','jpeg'],
    '21': ['flash','light']   # 9
}

#### feature 동의어 앞으로 추가해야함

In [48]:
db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT * FROM Features"
feature_db = pd.read_sql(SQL, db)
feature_db

,FeatureID,CategoryID,TeamID,FeatureName,FeatureNameRe,UserIP,LastUpdatedDate
0,12,2,0,resolution,(resolution)|(pixel)|(megapixel),None,None
1,13,2,0,lens,(lens)|(wide angle)|(normal range),None,None
2,14,2,0,optical,(optical)|(zoom)|(optical zoom)|(digital zoom),None,None
3,15,2,0,memory,(memory)|(megabytes)|(MB),None,None
4,16,2,0,burst,(burst)|(continuous)|(shutter)|(recovery)|(mot...,None,None
5,17,2,0,battery,(battery)|(batteries)|(power),None,None
6,18,2,0,focus,(focus)|(exposure)|(manual)|(iso),None,None
7,19,2,0,LCD,(LCD)|(screen),None,None
8,20,2,0,compression,(compression)|(compress)|(jpeg),None,None
9,21,2,0,flash,(flash)|(light),None,None


In [49]:
feature_db = feature_db[['FeatureID', 'FeatureName', 'FeatureNameRe']]

In [50]:
feature_db

,FeatureID,FeatureName,FeatureNameRe
0,12,resolution,(resolution)|(pixel)|(megapixel)
1,13,lens,(lens)|(wide angle)|(normal range)
2,14,optical,(optical)|(zoom)|(optical zoom)|(digital zoom)
3,15,memory,(memory)|(megabytes)|(MB)
4,16,burst,(burst)|(continuous)|(shutter)|(recovery)|(mot...
5,17,battery,(battery)|(batteries)|(power)
6,18,focus,(focus)|(exposure)|(manual)|(iso)
7,19,LCD,(LCD)|(screen)
8,20,compression,(compression)|(compress)|(jpeg)
9,21,flash,(flash)|(light)


In [51]:
FeatureNameRe={
    '1': '(resolution)|(pixel)|(megapixel)',
    '2': '(lens)|(wide angle)|(normal range)',
    '3': '(optical)|(zoom)|(optical zoom)|(digital zoom)',
    '4': '(memory)|(megabytes)|(MB)',
    '5': '(burst)|(continuous)|(shutter)|(recovery)|(motion)|(sport)',
    '6': '(battery)|(batteries)|(power)',
    '7': '(focus)|(exposure)|(manual)|(iso)|(ISO)',
    '8': '(LCD)|(screen)',
    '9': '(compression)|(compress)|(jpeg)',
    '10': '(flash)|(light)'
}

### for문으로 문장마다 feature label 붙이기

for문 four회

In [52]:
# 
# 중복되는 행 삭제 참조 : https://nittaku.tistory.com/124

# 임의로 index만드는.. 
index_list = ['0','1','2']
df = pd.DataFrame(columns=['review_index','sentence','FeatureName','FeatureNameRe'])


index_count = 0

for one_review in data['Review']:
    sentences = sent_tokenize(one_review)   
    
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        
        for token in tokens:
            for j in range(10):
                if token in feature[str(j+12)]:
                    df = df.append(pd.DataFrame([[index_count, sentence, j+12,feature_db['FeatureNameRe'][j] ]], columns=['review_index','sentence','FeatureName','FeatureNameRe']), ignore_index=True)

                else:
                    pass
    index_count += 1
    
    
# nan값을 가진 행을 삭제. 맨 처음에 index만들기 위해 임의로 만든 행을 삭제시키는 코드
df = df.dropna(axis=0)
# review인덱스와 feature이름이 중복되는 행을 제거함. keep인자는 중복 지우고 남길 행을 의미함. last이므로 마지막행만 남긴다는 의미
df = df.drop_duplicates(['review_index','FeatureName'], keep='last')  
# 인덱스 초기화
df = df.reset_index(drop=True)


In [53]:
df

,review_index,sentence,FeatureName,FeatureNameRe
0,0,It has a variety of features including action ...,16,(burst)|(continuous)|(shutter)|(recovery)|(mot...
1,0,It has a variety of features including action ...,13,(lens)|(wide angle)|(normal range)
2,0,"It's done for you automatically, and you can c...",21,(flash)|(light)
3,0,If you need parts (like rechargeable batteries...,17,(battery)|(batteries)|(power)
4,1,... after two previous attempts at buying my m...,17,(battery)|(batteries)|(power)
5,1,She's 68 and has a bit of difficulty with dext...,15,(memory)|(megabytes)|(MB)
6,1,It's very easy to learn and very light weight ...,21,(flash)|(light)
7,2,I like this camera so much that I bought two m...,12,(resolution)|(pixel)|(megapixel)
8,3,"I bought a second battery of course, but that ...",17,(battery)|(batteries)|(power)
9,3,"In camera red eye reduction is avialable, as i...",16,(burst)|(continuous)|(shutter)|(recovery)|(mot...


### class 사용... 안됨

In [11]:
# 이 셀은 쓰지 마세요


#### 줄이는 중


class runProgram :
    index_list = ['0','1','2']
    
    
    
    def __init__(self, reviews):
        self.reviews = reviews
        self.index_count = 0
        
        self.sentences = self.review2sentences(self.reviews)
        
        #######
        
        self.tokens = self.tokenizer(self.sentences)
        self.df = self.feature_labeler(self.tokens)
        self.df = self.fix_df(self.df)
        self.index_list = ['0','1','2']

#     @staticmethod
    def review2sentences(self, reviews):  
        for idx,one_review in enumerate(reviews):
            sentences = sent_tokenize(one_review)   
            return sentences


    def tokenizer(self,sentences):
        for sentence in sentences:
            tokens = word_tokenize(sentence)
            return tokens


    def feature_labeler(self,tokens):   
        df = pd.DataFrame(columns=['review_index','sentence','FeatureName','FeatureNameRe'])
        
        for token in tokens:
            for j in range(10):
                if token in feature[str(j)]:
                    df = df.append(pd.DataFrame([[idx, sentence, j+1,FeatureNameRe[str(j+1)] ]], columns=['review_index','sentence','FeatureName','FeatureNameRe']), ignore_index=True)

                else:
                    pass
        return df



    def fix_df(self,df):
        df = df.dropna(axis=0)
        df = df.drop_duplicates(['review_index','FeatureName'], keep='last')
        df = df.reset_index(drop=True)
        return df


#     def runProgram(reviews):
#         sentences = review2sentences(reviews)
#         tokens = tokenizer(sentences)
#         df = feature_labeler(tokens)
#         df = fix_df(df)

    def run() :
        for i in self.sentences :
            
        


SyntaxError: unexpected EOF while parsing (<ipython-input-11-0bbf29522ca0>, line 68)

In [ ]:
# a = runProgram(data['review'])

a.run()

In [ ]:
a.df

In [ ]:
data['review'][0]

# 1-2. db에서 feature전처리 끝낸 sentence 받아오기

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('sentence_final_190813.csv', encoding='utf-8', index_col = 0)

In [6]:
df = df.drop_duplicates(['Sentence','FeatureName'], keep='last') 

In [7]:
df = df.reset_index()

In [8]:
len(df)

439673

In [9]:
df.head()

,SentenceID,Sentence,title,FeatureName,brand,asin,ReviewID
0,2013768,"Only downside--AA batteries needed, but with r...",Kodak C180 10 MP HD Digital Camera with 3x Opt...,battery,NaN,B0025X1AKO,186353
1,2598459,I think they should have included an instructi...,Canon PowerShot ELPH 340 HS 16MP Digital Camer...,focus,Canon,B00HLDFNKQ,302137
2,2330478,I aready own the 7D with several lenses.,Canon EOS Rebel T3 12.2 MP CMOS Digital SLR Ca...,lens,Canon,B005LW580O,245675
3,2532464,I especially like the re-chargeable battery.Pl...,Canon PowerShot A2500 16MP Digital Camera with...,battery,Canon,B00B5HE1SY,286602
4,1999734,I bought the new Canon A2100 IS for 4 main rea...,Canon PowerShot A2100IS 12.1 MP Digital Camera...,optical,NaN,B001SER4AG,183925


## 3. KW 사전 구축하기

하지만 DB에 있는 사전을 쿼리문으로 가져오는 연습할 것

참조 : https://www.nltk.org/_modules/nltk/corpus/reader/comparative_sents.html

In [18]:
nltk.download('comparative_sentences')
from nltk.corpus import comparative_sentences

comparison = comparative_sentences.keywords_readme()

[nltk_data] Downloading package comparative_sentences to
[nltk_data]     C:\Users\seonmin\AppData\Roaming\nltk_data...
[nltk_data]   Package comparative_sentences is already up-to-date!


In [19]:
comparison

['POS tag JJR //adjective, comparative',
 'POS tag RBR //adverb, comparative',
 'POS tag JJS //adjective, superlative',
 'POS tag RBS //adjective, superlative',
 'as <word> as //e.g. as good as',
 'beat',
 'inferior',
 'outstrip',
 'both',
 'on par with',
 'Choice',
 'choose',
 'prefer',
 'recommend',
 'outperform',
 'superior',
 'all',
 'up against',
 'less',
 'favor',
 'defeat',
 'twice',
 'thrice',
 'half',
 'same',
 'either',
 'Compete',
 'Number one',
 'one of few',
 'more',
 'like',
 'behind',
 'similar',
 'identical',
 'Versus',
 'first',
 'outdistance',
 'before',
 'double',
 'outsell',
 'nobody',
 'Vs',
 'last',
 'after',
 'thrice',
 'improve',
 'equal',
 'equivalent',
 'together',
 'altogether',
 'Alternate',
 'only',
 'outmatch',
 'ahead',
 'fraction',
 'outdo',
 'match',
 'unmatched',
 'peerless',
 'differ',
 'One of few',
 'outwit',
 'rival',
 'alternate',
 'Compare',
 'top',
 'exceed',
 'lead',
 'win',
 'outstrip',
 'none',
 'near',
 'unrivaled',
 'dominate',
 'second',
 

In [20]:
KW = comparison[5:]

In [25]:
KW

['beat',
 'inferior',
 'outstrip',
 'both',
 'on par with',
 'Choice',
 'choose',
 'prefer',
 'recommend',
 'outperform',
 'superior',
 'all',
 'up against',
 'less',
 'favor',
 'defeat',
 'twice',
 'thrice',
 'half',
 'same',
 'either',
 'Compete',
 'Number one',
 'one of few',
 'more',
 'like',
 'behind',
 'similar',
 'identical',
 'Versus',
 'first',
 'outdistance',
 'before',
 'double',
 'outsell',
 'nobody',
 'Vs',
 'last',
 'after',
 'thrice',
 'improve',
 'equal',
 'equivalent',
 'together',
 'altogether',
 'Alternate',
 'only',
 'outmatch',
 'ahead',
 'fraction',
 'outdo',
 'match',
 'unmatched',
 'peerless',
 'differ',
 'One of few',
 'outwit',
 'rival',
 'alternate',
 'Compare',
 'top',
 'exceed',
 'lead',
 'win',
 'outstrip',
 'none',
 'near',
 'unrivaled',
 'dominate',
 'second',
 'nonpareil',
 'advantage',
 'unlike',
 'least',
 'outclass',
 'outfox',
 'outdistance',
 'most']

In [22]:
KW_list = comparison[5:]

# 4. KW로 SS / CS 구분하기

In [23]:
df['sentence']

KeyError: 'sentence'

In [98]:
POS_tag_list =['JJR','RBR','JJS','RBS']
df['SS/CS'] = None
index_count = 0


for sentence in df['Sentence']:
    tokens = word_tokenize(sentence)
    tokens_tag = nltk.pos_tag(tokens)

    for token in tokens_tag:
        if token[0] in KW_list or token[1] in POS_tag_list:
            df['SS/CS'][index_count] = 'CS'

#         elif token[1] in POS_tag_list:
#             df['SS/CS'][index_count] = 'CS'

        if df['SS/CS'][index_count] in ['CS']:
            continue

        else:
            df['SS/CS'][index_count] = 'SS'
    
    if index_count % 1000 == 0:
        print('현재까지 '+ str(index_count) + ' 번째 문장 완료')
    
    index_count += 1
    
    

C:\SEONMIN\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


현재까지 0 번째 문장 완료


C:\SEONMIN\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


현재까지 1000 번째 문장 완료
현재까지 2000 번째 문장 완료
현재까지 3000 번째 문장 완료
현재까지 4000 번째 문장 완료


KeyboardInterrupt: 

In [ ]:
df

In [ ]:
df[df['SS/CS'] == 'CS']

In [ ]:
df['Sentence'][1118]

# 5. MPQA 사전으로 POS, NEG 라벨링

In [58]:
db = pymysql.connect(
    host='ec2-13-209-223-88.ap-northeast-2.compute.amazonaws.com',  # DATABASE_HOST
    port=3306,
    user='nlpuser',  # DATABASE_USERNAME
    passwd='1234',  # DATABASE_PASSWORD
    db='nlp',  # DATABASE_NAME
    charset='utf8'
)

SQL = "SELECT * FROM TotalDic"
MPQA = pd.read_sql(SQL, db)
MPQA

,KeywordID,CategoryID,TeamID,DicType1,DicType2,Rule,Label,UserIP,LastUpdatedDate
0,1,2,6,Sentiment,MPQA,a+,POS,None,None
1,2,2,6,Sentiment,MPQA,abound,POS,None,None
2,3,2,6,Sentiment,MPQA,abounds,POS,None,None
3,4,2,6,Sentiment,MPQA,abundance,POS,None,None
4,5,2,6,Sentiment,MPQA,abundant,POS,None,None
5,6,2,6,Sentiment,MPQA,accessable,POS,None,None
6,7,2,6,Sentiment,MPQA,accessible,POS,None,None
7,8,2,6,Sentiment,MPQA,acclaim,POS,None,None
8,9,2,6,Sentiment,MPQA,acclaimed,POS,None,None
9,10,2,6,Sentiment,MPQA,acclamation,POS,None,None


In [59]:
MPQA = MPQA[MPQA['DicType2'] == 'MPQA']
MPQA_list = MPQA[['Rule','Label']]
POS_list = MPQA_list[MPQA_list['Label']=='POS']
pos_list = POS_list['Rule']
pos_list = list(pos_list)
NEG_list = MPQA_list[MPQA_list['Label']=='NEG']
neg_list = NEG_list['Rule']
neg_list = list(neg_list)

In [60]:
pos_list

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [61]:
neg_list

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [ ]:
df['NumOfPOS'] = None
df['NumOfNEG'] = None
df['POS_token'] = None
df['NEG_token'] = None
df['Label'] = None


index_count = 0

for sentence in df['Sentence']:
    tokens = word_tokenize(sentence)
    pos_count = 0
    neg_count = 0
    pos_token_list = []
    neg_token_list = []

    for token in tokens:
        if token in pos_list:
            pos_count += 1
            pos_token_list.append(token)
            

        elif token in neg_list:
            neg_count += 1
            neg_token_list.append(token)
               
        else:
            pass

    df['NumOfPOS'][index_count] = pos_count
    df['POS_token'][index_count] = pos_token_list    # 같은 token이 있어도 중복시킨다
    df['NumOfNEG'][index_count] = neg_count
    df['NEG_token'][index_count] = neg_token_list
    
    if pos_count > neg_count:
        df['Label'][index_count] = 'POS'
        
    elif pos_count < neg_count:
        df['Label'][index_count] = 'NEG'
    
    else:
        pass
    
    index_count += 1


In [ ]:
df

In [64]:
# df.to_csv('to국승지.csv', index=0)

In [ ]:
----------------------------------------

In [3]:
product_id = pd.read_csv('product_name_number.csv', encoding='utf-8')

In [4]:
product_id

,model,count
0,sl1,5
1,d5000,4
2,d3100,4
3,c190,3
4,d60,3
5,dv300f,3
6,st200f,3
7,20d,3
8,t3i,3
9,x018,3


In [8]:
product_list = list(product_id['model'])

In [9]:
product_list

['sl1',
 'd5000',
 'd3100',
 'c190',
 'd60',
 'dv300f',
 'st200f',
 '20d',
 't3i',
 'x018',
 'k-x',
 'l820',
 'xsi',
 'd90',
 'z915',
 'xti',
 'c180',
 'dc4800',
 'wb100',
 'zr400',
 't99',
 'c143',
 'd7100',
 's3300',
 'z3',
 'm583',
 'd-lux',
 'sx150',
 's5',
 'd70',
 'd300',
 'vg-180',
 'a7',
 't3',
 'f660exr',
 'i834',
 'zr1000',
 'nex-3nl',
 'tl220',
 'd5200',
 'jx500',
 'wg-2',
 't5i',
 '1ds',
 'xs',
 'v705',
 '60d',
 '7022',
 's5200',
 'e-p2',
 'd3200',
 'd5100',
 'p500',
 'sipix',
 'nex-3',
 'z5120',
 '5d',
 'k-r',
 'dmc-lz',
 'v7028',
 'c-2040',
 'e8',
 'p310',
 't135',
 'z1085',
 'dmc-zr1',
 'dmc-tz',
 's3600',
 's51',
 'wx5',
 'dmc-fx35a',
 'sz-16',
 't2i',
 'sp-560',
 'c18',
 'l28',
 '3000z',
 'v1233',
 'd-460',
 's425',
 'rx100m',
 'dsc-tx',
 'd2xs',
 'f300ex',
 'fe3',
 'z10fd',
 'concepts vga',
 'a330l',
 'dmc-fz100',
 's61',
 'm547',
 'x-e1',
 'sx500',
 'dsc-t500',
 'e1486t',
 'dmc-fx58',
 'd3x',
 'dsc-t77',
 'vr-34',
 'e42',
 'mini',
 't730',
 'dsc-w290',
 'dsc-h20/b',


In [ ]:
---------------------------------------------------

In [26]:
s = pd.Series(['cat','hat','dog','fog','pet','fat'])

In [27]:
searchfor = ['og', 'at']

In [28]:
s[s.str.contains('|'.join(searchfor))]

0    cat
1    hat
2    dog
3    fog
5    fat
dtype: object

In [29]:
df_test = df[:1000]

In [31]:
df_test['Sentence']

0      Only downside--AA batteries needed, but with r...
1      I think they should have included an instructi...
2               I aready own the 7D with several lenses.
3      I especially like the re-chargeable battery.Pl...
4      I bought the new Canon A2100 IS for 4 main rea...
5      I bought the new Canon A2100 IS for 4 main rea...
6      I have always been happy with Canons and my Po...
7      Zoom is important to me and getting 6x in a sm...
8      The only very small drawback is that it takes ...
9      The picture quality is great, and with the mus...
10     You can take great shots, day and night, as lo...
11     The 51 point AF system is awesome if you have ...
12     It is fast and has the horsepower to meet the ...
13     It actually fit my hands better.Lastly, having...
14     I loved having the ability to dial in the expo...
15     I loved having the ability to dial in the expo...
16     I set the exposure for the ambient conditions ...
17     I set the exposure for t

In [37]:
KW_list

['beat',
 'inferior',
 'outstrip',
 'both',
 'on par with',
 'Choice',
 'choose',
 'prefer',
 'recommend',
 'outperform',
 'superior',
 'all',
 'up against',
 'less',
 'favor',
 'defeat',
 'twice',
 'thrice',
 'half',
 'same',
 'either',
 'Compete',
 'Number one',
 'one of few',
 'more',
 'like',
 'behind',
 'similar',
 'identical',
 'Versus',
 'first',
 'outdistance',
 'before',
 'double',
 'outsell',
 'nobody',
 'Vs',
 'last',
 'after',
 'thrice',
 'improve',
 'equal',
 'equivalent',
 'together',
 'altogether',
 'Alternate',
 'only',
 'outmatch',
 'ahead',
 'fraction',
 'outdo',
 'match',
 'unmatched',
 'peerless',
 'differ',
 'One of few',
 'outwit',
 'rival',
 'alternate',
 'Compare',
 'top',
 'exceed',
 'lead',
 'win',
 'outstrip',
 'none',
 'near',
 'unrivaled',
 'dominate',
 'second',
 'nonpareil',
 'advantage',
 'unlike',
 'least',
 'outclass',
 'outfox',
 'outdistance',
 'most']

In [35]:
test = df_test['Sentence'][df_test['Sentence'].str.contains('|'.join(KW_list))]

In [39]:
test

0      Only downside--AA batteries needed, but with r...
3      I especially like the re-chargeable battery.Pl...
4      I bought the new Canon A2100 IS for 4 main rea...
5      I bought the new Canon A2100 IS for 4 main rea...
7      Zoom is important to me and getting 6x in a sm...
8      The only very small drawback is that it takes ...
9      The picture quality is great, and with the mus...
10     You can take great shots, day and night, as lo...
11     The 51 point AF system is awesome if you have ...
12     It is fast and has the horsepower to meet the ...
13     It actually fit my hands better.Lastly, having...
16     I set the exposure for the ambient conditions ...
17     I set the exposure for the ambient conditions ...
18     I set the exposure for the ambient conditions ...
19     Even with the ambient light changing each minu...
20     Even with the ambient light changing each minu...
21     Even with the ambient light changing each minu...
22     My biggest issues with t